
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# プーリング

:label: `sec_pooling`

多くの場合、私たちの最終的なタスクでは、画像に関する何らかのグローバルな質問が行われます (たとえば、*画像には猫が含まれていますか?)。*したがって、最終層のユニットは入力全体に敏感でなければなりません。情報を徐々に集約して、ますます粗いマップを生成することで、処理の中間層で畳み込み層の利点をすべて維持しながら、最終的にグローバル表現を学習するというこの目標を達成します。ネットワークの奥深くに行けば行くほど、各隠れノードが感知する受容野（入力に対して）が大きくなります。空間解像度を下げると、コンボリューション カーネルがより大きな有効領域をカバーするため、このプロセスが加速されます。

さらに、エッジなどの下位レベルの特徴を検出する場合 ( :numref: `sec_conv_layer`で説明したように)、表現が平行移動に対してある程度不変であることが必要になることがよくあります。たとえば、白と黒の間の輪郭がはっきりしている画像`X`を取得し、画像全体を右に 1 ピクセル分シフトすると、つまり`Z[i, j] = X[i, j + 1]` 、出力は次のようになります。新しいイメージ`Z`は大きく異なる可能性があります。エッジは 1 ピクセルずつ移動します。実際には、オブジェクトがまったく同じ場所に存在することはほとんどありません。実際、三脚と静止した物体を使用した場合でも、シャッターの動きによるカメラの振動により、すべてが 1 ピクセル程度ずれる可能性があります (ハイエンド カメラには、この問題に対処するための特別な機能が搭載されています)。

このセクションでは、位置に対する畳み込み層の感度を軽減することと、表現を空間的にダウンサンプリングするという 2 つの目的を果たす*プーリング層*を紹介します。


In [1]:
import torch
from torch import nn
from d2l import torch as d2l


## 最大プーリングと平均プーリング

畳み込み層と同様に、*プーリング*オペレーターは、ストライドに従って入力内のすべての領域上をスライドする固定形状のウィンドウで構成され、固定形状のウィンドウ (*プーリング ウィンドウ*とも呼ばれる) が通過する各位置に対して 1 つの出力を計算します。ただし、畳み込み層の入力とカーネルの相互相関計算とは異なり、プーリング層にはパラメーターが含まれません (*カーネル*がありません)。代わりに、プーリング演算子は決定的であり、通常はプーリング ウィンドウ内の要素の最大値または平均値を計算します。これらの操作は、それぞれ*最大プーリング*(略して*max-pooling* ) および*平均プーリング*と呼ばれます。

*平均プーリング*は基本的に CNN と同じくらい古いものです。このアイデアは画像のダウンサンプリングに似ています。低解像度の画像の 2 番目 (または 3 番目) ごとのピクセルの値を単に取得するのではなく、複数の隣接するピクセルからの情報を組み合わせているため、隣接するピクセルを平均して、信号対雑音比がより優れた画像を取得できます。 *Max-pooling は、*認知神経科学の文脈で :citet: `Riesenhuber.Poggio.1999`で導入され、物体認識の目的で情報の集約がどのように階層的に集約されるかを説明しました。また、音声認識における初期のバージョンは :cite: `Yamaguchi.Sakamoto.Akabane.ea.1990` 。ほとんどの場合、max-pooling とも呼ばれる方法が推奨されます。

どちらの場合も、相互相関演算子の場合と同様に、プーリング ウィンドウは入力テンソルの左上から始まり、入力テンソルを左から右、上から下にスライドすると考えることができます。プーリング ウィンドウがヒットする各位置で、最大プーリングが使用されるか平均プーリングが使用されるかに応じて、ウィンドウ内の入力サブテンソルの最大値または平均値が計算されます。 

![](http://d2l.ai/_images/pooling.svg) :label: `fig_pooling`

 :numref: `fig_pooling`の出力テンソルの高さは 2、幅は 2 です。4 つの要素は、各プーリング ウィンドウの最大値から導出されます。

 $$ \max(0, 1, 3, 4)=4,\ \max(1, 2, 4, 5)=5,\ \max(3, 4, 6, 7)=7,\ \max( 4、5、7、8)=8.\ $$

より一般的には、上記のサイズの領域を集約することで $p \times q$ プーリング層を定義できます。エッジ検出の問題に戻ると、畳み込み層の出力を $2\times 2$ max-pooling の入力として使用します。畳み込み層の入力を`X`で表し、プーリング層の出力を`Y`で表します。 `X[i, j]` 、 `X[i, j + 1]` 、 `X[i+1, j]` 、および`X[i+1, j + 1]`の値が異なるかどうかに関係なく、プーリング層は常に`Y[i, j] = 1`を出力します。つまり、$2\times 2$ max-pooling 層を使用しても、畳み込み層によって認識されたパターンが高さまたは幅で 1 要素しか移動しないかどうかを検出できます。

以下のコードでは、 `pool2d`関数に**プーリング層の順伝播を実装します**。この関数は、 :numref: `sec_conv_layer`の`corr2d`関数に似ています。ただし、入力内の各領域の最大値または平均として出力を計算するカーネルは必要ありません。


In [2]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y


**[2 次元最大プーリング層の出力を検証する**] ために、:numref: `fig_pooling`で入力テンソル`X`を構築できます。


In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])


また、(**平均プーリング層**) についても実験します。


In [4]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])


## [**パディングとストライド**]

畳み込み層と同様に、プーリング層は出力形状を変更します。また、以前と同様に、入力をパディングしてストライドを調整することで、目的の出力形状を実現するように操作を調整できます。深層学習フレームワークの組み込みの 2 次元最大プーリング層を介して、プーリング層でのパディングとストライドの使用を実証できます。まず、形状が 4 次元で、例の数 (バッチ サイズ) とチャネルの数が両方とも 1 である入力テンソル`X`を構築します。


In [5]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])


プーリングはエリアからの情報を集約するため、(**深層学習フレームワークはデフォルトでプーリング ウィンドウ サイズとストライドを一致させます)。**たとえば、形状`(3, 3)`のプーリング ウィンドウを使用すると、次のようにして`(3, 3)`のストライド形状が得られます。デフォルト。


In [6]:
pool2d = nn.MaxPool2d(3)
# Pooling has no model parameters, hence it needs no initialization
pool2d(X)

tensor([[[[10.]]]])


予想通り、[**ストライドとパディングは手動で指定できます**] 必要に応じてフレームワークのデフォルトをオーバーライドできます。


In [7]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])


もちろん、以下の例に示すように、それぞれ任意の高さと幅を持つ任意の長方形のプーリング ウィンドウを指定できます。


In [8]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])


## 複数のチャンネル

マルチチャネル入力データを処理する場合、**畳み込み層のようにチャネル上の入力を合計するのではなく、[プーリング層は各入力チャネルを個別にプールします**]。これは、プーリング層の出力チャネルの数が入力チャネルの数と同じであることを意味します。以下では、テンソル`X`と`X + 1`をチャネル次元で連結して、2 チャネルの入力を構築します。


In [9]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])


見てわかるように、プール後も出力チャネルの数は 2 のままです。


In [10]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])


## まとめ

プーリングは非常に簡単な操作です。これは、その名前が示すとおり、値のウィンドウ全体で結果を集計します。ストライドやパディングなどのすべての畳み込みセマンティクスは、以前と同じ方法で適用されます。プーリングはチャネルとは無関係であることに注意してください。つまり、チャネルの数は変更されず、各チャネルに個別に適用されます。最後に、2 つの一般的なプーリングの選択肢のうち、出力にある程度の不変性を与えるため、平均プーリングよりも max-pooling の方が適しています。一般的な選択は、出力の空間解像度を 4 分の 1 にするために $2 \times 2$ のプーリング ウィンドウ サイズを選択することです。

プーリング以外にも解像度を下げる方法はたくさんあることに注意してください。たとえば、確率的プーリング :cite: `Zeiler.Fergus.2013`およびfractional max-pooling :cite: `Graham.2014`では、集計がランダム化と組み合わされます。これにより、場合によっては精度がわずかに向上することがあります。最後に、後でアテンション メカニズムで説明するように、クエリと表現ベクトルの間のアライメントを使用するなど、出力を集約するさらに洗練された方法があります。

## 演習
1. 畳み込みを通じて平均プーリングを実装します。
1. 最大プーリングは畳み込みだけでは実装できないことを証明します。
1. 最大プーリングは、ReLU 操作、つまり $\mathrm{ReLU}(x) = \max(0, x)$ を使用して実現できます。<ol><li> ReLU 演算のみを使用して $\max (a, b)$ を表現します。
1. これを使用して、畳み込みと ReLU 層による最大プーリングを実装します。
1.  $2 \times 2$ の畳み込みには、いくつのチャネルとレイヤーが必要ですか? $3 \times 3$ の畳み込みの数。



[ディスカッション](https://discuss.d2l.ai/t/72)
